In [1]:
import os

os.environ["OPENAI_API_KEY"] = "sk-socialthingum-Lzt6aXn0b7k2ia0tjcmqT3BlbkFJLWlJZwLsUuKQ6OSfzwgC"

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

### First tentative: general idea

In [3]:
prompt_system = """
You are a helpful and creative assistant specialized in cooking and recipe generation.

The user will provide either:
- A list of ingredients
- A recipe title

Your task is to generate a complete and coherent recipe based on the input. The recipe should include:
- A descriptive and fitting title
- A list of ingredients
- Step-by-step preparation instructions

If the input is a list of ingredients, invent a suitable dish using them. If the input is a title, develop a recipe that matches the name.

Be clear, concise, and make sure the result is easy to follow for home cooks.

output:
{{
    "title": "recipe title",
    "description": "recipe description",
    "list_of_ingredients": ["ingredient 1", "ingredient 2", ...],
    "list_of_steps": ["step 1", "step 2", ...],
}}
"""


In [4]:
response_schemas = [
    ResponseSchema(
        name='title',
        description='The title of the recipe',
    ),
    ResponseSchema(
        name='description',
        description='A short description of the recipe',
    ),
    ResponseSchema(
        name='list_of_ingredients',
        description='A list of ingredients',
    ),
    ResponseSchema(
        name='list_of_steps',
        description='A list of steps to prepare the dish, only the list',
    )
]

In [5]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [6]:
query_simplification_prompt = ChatPromptTemplate.from_messages(
    [('system', prompt_system), ('user', '{messages}')]
).partial(format_instructions=output_parser.get_format_instructions())


In [7]:
user_input = input("Enter a recipe title or a list of ingredients (comma-separated): ")

In [8]:
# Use the updated ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Create the final chain using the new syntax
recipe_chain = query_simplification_prompt | llm | output_parser

# Run using .invoke instead of .run
response = recipe_chain.invoke({"messages": user_input})


In [9]:
print("Title:", response['title'])
print("Description:", response['description'])
print("\nIngredients:")
for ingredient in response['list_of_ingredients']:
    print("-", ingredient)

print("\nPreparation Steps:")
for i, step in enumerate(response['list_of_steps'], 1):
    print(f"{i}. {step}")

Title: Simple Roasted Chicken
Description: This is a classic recipe for a juicy, flavorful roasted chicken. It's easy to prepare with minimal ingredients and makes for a perfect Sunday dinner. Serve with your favorite sides for a wholesome meal.

Ingredients:
- 1 whole chicken (about 4 lbs)
- 2 tbsp olive oil
- 1 tbsp salt
- 1/2 tbsp black pepper
- 1 lemon, halved
- 4 cloves garlic, peeled
- Several sprigs of fresh thyme

Preparation Steps:
1. Preheat your oven to 425°F (220°C).
2. Rinse the chicken inside and out under cold running water. Dry thoroughly with paper towels.
3. Rub the chicken all over with the olive oil.
4. Season the chicken inside and out with salt and pepper.
5. Stuff the cavity of the chicken with the halved lemon, garlic cloves, and thyme sprigs.
6. Place the chicken in a roasting pan breast side up. Roast in the preheated oven for 1 hour and 15 minutes, or until the juices run clear when you cut between a leg and thigh.
7. Remove the chicken from the oven and let 

### List of ingredients promts

In [10]:
prompt_system_ingrs = """
You are a creative and helpful cooking assistant.

The user will provide a list of available ingredients.  
Your task is to create a complete and realistic recipe using **only those ingredients** — do not invent or add anything not included in the list.

The recipe must include:
- A clear and descriptive title
- Step-by-step preparation instructions

Important rules:
- Do NOT use any ingredient that is not listed.
- If a necessary item (e.g., salt, oil, water) is missing, do NOT assume it's available — exclude it.
- You can adjust the form or quantity of the ingredients (e.g., chop, grate, blend), but not add new ones.
- If the ingredients are unusual together, invent a plausible or creative dish that still makes good use of them.

Output your response in the following JSON format:

{{
    "title": "recipe title",
    "list_of_steps": ["step 1", "step 2", ...]
}}

"""

In [11]:
response_schemas_2 = [
    ResponseSchema(
        name='title',
        description='The title of the recipe',
    ),
    ResponseSchema(
        name='list_of_steps',
        description='A list of steps to prepare the dish, only the list',
    )
]

In [12]:
output_parser_2 = StructuredOutputParser.from_response_schemas(response_schemas_2)

In [13]:
prompt_system_ingrs_ = ChatPromptTemplate.from_messages(
    [('system', prompt_system_ingrs), ('user', '{messages}')]
).partial(format_instructions=output_parser_2.get_format_instructions())

In [14]:
user_input_2 = input("Enter a list of ingredients (comma-separated): ")

In [15]:
# Use the updated ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Create the final chain using the new syntax
recipe_chain_2 = prompt_system_ingrs_ | llm | output_parser_2

# Run using .invoke instead of .run
response_2 = recipe_chain_2.invoke({"messages": user_input_2})


In [16]:
print("Title:", response_2['title'])


print("\nPreparation Steps:")
for i, step in enumerate(response_2['list_of_steps'], 1):
    print(f"{i}. {step}")


Title: Garlic Tomato Chicken

Preparation Steps:
1. Step 1: Wash and pat dry the chicken.
2. Step 2: Heat the oil in a pan over medium heat.
3. Step 3: Add the chicken to the pan and cook until it is browned and cooked through, about 4-6 minutes per side depending on thickness. Remove the chicken from the pan and set it aside.
4. Step 4: In the same pan, add the garlic and cook until it is fragrant, about 30 seconds.
5. Step 5: Add the tomatoes to the pan, cook until they are soft and the juices are released, about 5-7 minutes.
6. Step 6: Return the chicken to the pan, nestling it into the tomatoes. Let it cook for another 2-3 minutes to absorb the flavors.
7. Step 7: Serve the garlic tomato chicken while it's hot.
